<p><img alt="Colaboratory logo" height="150px" src="https://blog-static.infra.grancursosonline.com.br/wp-content/uploads/2015/09/03174643/UFC2.png" align="right" hspace="30px" vspace="0px"></p>  

<h1>
<strong> Universidade Federal do Ceará </strong>  <br>
Bacheralado em Matemática Industrial  <br>
Primeiro Projeto Computacional - 27/10/ 2020  <br>
Disciplina: Programação Não Linear  <br>
Professor: Ricardo Coelho  <br>
Grupo: 02 <br>
Membros: Daniel Nunes, Yuri Martins <br>
Trabalho: Métodos de Otimização Restrita
</h1>


<h2> <strong> 1 - Resumo dos fundamentos teóricos </strong> </h2>



<h3> <strong> 1.1 - Métodos de Barreira </strong> </h3>

Na otimização restrita, um campo da matemática, uma função de barreira é uma função contínua cujo valor em um ponto aumenta até o infinito conforme o ponto se aproxima do limite da região viável de um problema de otimização. Essas funções são usadas para substituir as restrições de desigualdade por um termo penalizador na função objetivo que é mais fácil de manipular.  

Os dois tipos mais comuns de funções de barreira são funções de barreira inversa e funções de barreira logarítmica.

<strong> Motivação -  </strong>
Considere o seguinte problema de otimização restrita:  

Minimizar $f(x)$  
Sujeito a $h(x) >= 0$  


A ideia do método é transformar um problema restrito em um problema irrestrito, com uma função de barreira.

Existe duas formas de criar essa função barreira

- Barreira logarítimica:
$$
\psi(x) = - \sum_{i=1}^{m} log(- g_i(x))
$$
- Barreira inversa
$$
\psi(x) = - \sum_{i=1}^{m} \frac{1}{g_i(x) } 
$$

Note que esse método resolve problemas de otimização sujeito somente a restrições de desigualdade.  

Além disso, a sequência de pontos obtidos pelo método necessitam ser viáveis.  
Agora considere uma família de funções:  
$$
\phi(x, t) = f(x) + t \psi(x)
$$

sendo $t > 0$ o parâmetro penalizador, e o problema de otimização restrito pode ser reescrito da seguinte maneira:

$$
\min \phi(x, t)
$$

<strong> (Algoritmo do Método de Barreira) </strong>  
Escolha uma função barreira $\phi$  
Escolha $t_0 > 0$ e $k = 0$  

1 - Calcule $x^k \in \mathbb{R}^n$ como uma solução do problema com barreira, onde a função ́é dada por φ com $c = t^k$ ;  

2 - Se o ponto encontrado atende ao critério de parada, PARE. Caso contrário, SIGA;  

3 - Escolha $t_{k+1}$ < $ t_k$ e tome $k = k + 1$ Retome ao passo 1  


<h3> <strong> 1.2 - Método de Penalidade </strong> </h3>

<strong> Motivação </strong> <br>

Transformar um problema de otimização restrita em um problema de otimizazação irrestrita.  

A ideia principal  ́e gerar uma sequência de pontos viáveis até convergir para a solução do problema de otimização restrito.  

Considere o problema de minimização com a função objetivo $f: \mathbb{R}^n \rightarrow  \mathbb{R} $ e $D \subset \mathbb{R}^n$ um conjunto qualquer.

Assim, definimos uma função $\psi$:
$$
\psi (x)  = \left\{\begin{matrix}
=0 & \forall x \in D\\ 
>0 &  \forall x \in \mathbb{R}^n- D
\end{matrix}\right.
$$

Esse termo est ́a bem definido, pois caso o conjunto D seja um conjunto viável formado
de restrições mistas, com restrições de igualdade e desigualdade, é coveniente definir.

$$
\Psi (x) = (h(x), max(0, g_1 (x), ... , max(0, g_m (x)  )
$$
$$
\psi (x) = || \Psi||^p
$$

Agora, considere uma família de probelmas irrestritos:
$$
min \: \: \phi (x, c) = f(x) + c \psi (x)
$$

Onde c > 0 é parâmetro penalizador.  


<strong> (Algoritmo do Método de Penalidade) </strong>  
Escolha uma função penalidade $\phi$  
Escolha $c_0 > 0$ e $k = 0$  

1 - Calcule $x^k \in \mathbb{R}^n$ como uma solução do problema com penalidade, onde a função ́é dada por φ com $c = c^k$ ;  

2 - Se o ponto encontrado atende ao critério de parada, PARE. Caso contrário, SIGA;  

3 - Escolha $c_{k+1}$ > $ c_k$ e tome $k = k + 1$ Retome ao passo 1  




<h3> <strong> 2 - Listagem do código fonte </strong> </h3>

In [ ]:
import numpy as np
from scipy import optimize

In [ ]:
def barreira(f, x0, erro=1e-8, k_max=10**3):
    t = 1
    k = 1
    x_opt_old = np.array(x0)
    while True:
        x_opt = optimize.fmin_bfgs(f, x0, args=(t,), disp=0)
        if t <= erro or np.linalg.norm(x_opt - x_opt_old) <= erro:
            return x_opt
        
        x_opt_old = np.copy(x_opt)
        t /= 10
        
        if k > k_max:
            break
        k += 1
    return np.inf

In [ ]:
def metodo_penalidade3(funcao, x, rest_igual=None, rest_desigual=None,  iter_max=1000, eps=1e-7):  
  c = 1
  k = 0
  min = 9999999999999
  min_old = 0
  if rest_igual == None:
    def func_pen(x):
      #print('desigual')
      return funcao(x) + c*(np.linalg.norm([max(0, g2(x)) for g2 in rest_desigual]))
  elif rest_desigual == None:
    def func_pen(x):
      #print('igual')
      return funcao(x) + c*(np.linalg.norm([g(x) for g in rest_igual]))
  else:
    def func_pen(x):
      #print('ambos')
      return funcao(x) + c*(np.linalg.norm(np.concatenate([[g(x) for g in rest_igual], [max(0, g2(x)) for g2 in rest_desigual]])))

  while True:
    if k > iter_max:
      return x, funcao(x)
    if abs(min - min_old) < eps:
      return x, funcao(x)  
    
    min_old = min
    x_old = x
    x = optimize.minimize(func_pen, x).x
    c = c + 10
    k = k + 1
    min = func_pen(x)
    #print(abs(min - min_old))
    """
    print({'K = ': k,
           'X = ': x,
           'C = ': c,
           'Min = ': func_pen(x)})"""
  return x, funcao(x)

Dados que ser ̃ao usado para testes do m ́etodo de Barreira:
